In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
! pip install -q kaggle
#Create a new API and import that as the file
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
zip_path = '/content/drive/MyDrive/Kaggle/cell-images-for-detecting-malaria.zip'
os.getcwd()
!cp '{zip_path}' .
!unzip -q 'cell-images-for-detecting-malaria.zip'
os.listdir()
data_dir = "/content/cell_images/cell_images"

In [ ]:
dim = 150
batch = 32

datagen =ImageDataGenerator(rescale=1/255.0, validation_split=0.3,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True)

train_data =datagen.flow_from_directory(data_dir, target_size=(dim,dim), batch_size=batch, class_mode = 'categorical', subset = 'training')
validation_data =datagen.flow_from_directory(data_dir, target_size=(dim,dim), batch_size=batch, class_mode = 'categorical', subset = 'validation', shuffle=False)

GradCam

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Load the saved model
model_path = "/content/malaria_model.h5"
model = keras.models.load_model(model_path)

# Grad-CAM implementation
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Define the last convolutional layer name
last_conv_layer_name = "conv2d_7"

# Path to the image you want to visualize
img_path = "/content/Uninfected.png"

# Display the original image
display(Image(img_path))

# Preprocess the image
img_array = get_img_array(img_path, size=(dim, dim))
img_array = img_array / 255.0

# Generate the Grad-CAM heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display the heatmap
plt.matshow(heatmap)
plt.show()

# Code for Grad-CAM superimposition
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on the original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad-CAM
    display(Image(cam_path))

# Path to save the Grad-CAM superimposed image
cam_path = "/content/cam.jpg"

# Save and display the Grad-CAM superimposed image
save_and_display_gradcam(img_path, heatmap, cam_path)

In [ ]:
model.summary()

SHAP


In [ ]:
!pip install shap

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import json
import shap

#Load your trained model from the saved file
model = tf.keras.models.load_model("/content/malaria_model.h5")

#Convert model to a function
def f(X):
    tmp = X.copy()
    tmp /= 255.0  # Normalize the input images to the range [0, 1]
    return model(tmp)

# Load the custom image from a file input
custom_image_path = "/content/Uninfected.png"  # Replace with the path to your custom image
custom_image = load_img(custom_image_path, target_size=(dim, dim))
custom_image = img_to_array(custom_image)
custom_image = np.expand_dims(custom_image, axis=0)

# Create a binary masker
masker = shap.maskers.Image("inpaint_telea", custom_image[0].shape)

# Use the SHAP explainer
explainer = shap.Explainer(f, masker, output_names=["Parasitized", "Uninfected"])

# Generate SHAP values for the custom image
shap_values_custom = explainer(custom_image, max_evals=5000, batch_size=5, outputs=shap.Explanation.argsort.flip[:1])

# Ensure SHAP values have the correct shape
if len(shap_values_custom.shape) == 3:
    shap_values_custom = shap_values_custom.squeeze()

# Visualize the SHAP values for the custom image
shap.image_plot(shap_values_custom, -custom_image)  # Note the negative sign to convert the pixel values to be suitable for plotting